In [1]:
# !pip install SQLAlchemy
# !pip install pyspark
# !pip install jupysql duckdb duckdb-engine
# !pip install delta-spark
# !pip install py4j
# !pip install -q findspark
# conda install -c conda-forge submitit 
# conda install corn, nbconvert
# !pip install graphviz
# !pip install hydra-core
# !pip install hydra-submitit-launcher
# !pip install submitit
# !pip install covalent-slurm-plugin
# !pip install ipynb-py-convert

In [2]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.24.0.8-3.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.24.0.8-3.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.24.0.8-3.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.24.0.8-3.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [3]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

# import submitit

import time as t 
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "20g")\
       .config("spark.driver.memory", "100g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
spark.sql("set spark.sql.execution.arrow.enabled=false")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/03 00:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/03 00:45:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/03 00:45:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/09/03 00:45:35 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


DataFrame[key: string, value: string]

In [4]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data.csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")

24/09/03 00:45:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df_population = spark.sql("SELECT * FROM population")

In [6]:
calculated_features = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [25]:
def whether_thisVIN_data_already_generated(thisVIN): 
    df_VINs_data = spark.sql("SELECT * FROM VINs_data")
    df_VINs_data = df_VINs_data.toDF("VIN", "TOTAL_ROWS")
    df_VINs_data_p = df_VINs_data.toPandas()
    for index, row in df_VINs_data_p.iterrows():
        if row['VIN'] == thisVIN:
          return 1
    return 0 

# Function to submit a SLURM job and return the job ID
def submit_job(script_path, thisVIN):
    submit_command = f"sbatch {script_path} {thisVIN}"
    print("The submit command for the submitted job is: ")
    print(submit_command)
    result = subprocess.run(submit_command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        job_id = re.findall(r'\d+', result.stdout)[0]
        print(f"Job {job_id} submitted successfully.")
        return job_id
    else:
        print("Job submission failed.")
        return None

# Function to check the status of multiple jobs
def check_jobs_status(job_ids):
    jobs_running = set(job_ids)
    while jobs_running:
        for job_id in list(jobs_running):
            status_command = f"squeue -j {job_id}"
            result = subprocess.run(status_command, shell=True, capture_output=True, text=True)
            if not result.stdout:  # If the job is no longer in the queue
                print(f"Job {job_id} completed.")
                jobs_running.remove(job_id)
        if jobs_running:
            print("Some jobs are still running...")
            print(jobs_running)
            t.sleep(1)  # Check every 5 seconds

# Function to retrieve job outputs
def get_jobs_output(output_files):
    for output_file in output_files:
        try:
            with open(output_file, "r") as file:
                output = file.read()
            print(f"Output from {output_file}:")
            print(output)
        except FileNotFoundError:
            print(f"Output file {output_file} not found.")
            



input_file_path = '/storage/home/yqf5148/work/volvoPennState/data/dataset/cleanedNormalized_resultedData.csv'


all_columns_names = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s] + calculated_features

resultedData_df = pd.read_csv(input_file_path, sep=',', names=all_columns_names, header=None)

# Step 1: Identify VINs with any calendar_day value higher than 1814
vin_with_high_calendar_day = resultedData_df[resultedData_df['calendar_day'] > 1814]['VIN'].unique()


# Convert vin_with_high_calendar_day to a set for fast lookup
vin_set = set(vin_with_high_calendar_day)

# Create an empty list to collect rows that should be kept
rows_to_keep = []

# Iterate over the DataFrame rows
for index, row in resultedData_df.iterrows():
    # Check if the VIN is not in the set
    if row['VIN'] not in vin_set:
        rows_to_keep.append(row)

print(rows_to_keep)        
# Convert the list of rows back to a DataFrame
filtered_resultedData_df = pd.DataFrame(rows_to_keep, columns=resultedData_df.columns).reset_index()
display(filtered_resultedData_df)

# Step 2: Add 731 to the 'calendar_day' values for these VINs
filtered_resultedData_df['calendar_day'] += 731

# If you want to keep the rest of the DataFrame intact and only modify these specific rows:
# Replace the original rows in the original DataFrame
# resultedData_df.update(filtered_resultedData_df)



limit_number_of_jobs = 30



# # Submitting multiple jobs
# job_ids = []
# output_files = []
# for index, row in df_pop_VINs_p.head(limit_number_of_jobs)iterrows():
#     thisVIN = row['VIN']

#     print(" VIN = {}'s data is being generated.".format(thisVIN))
#     aCurrentVIN = thisVIN 
#     df_filtered_population_for_this_VIN = df_all_pop_VINs.filter(f.col('VIN')==aCurrentVIN)
#     script_path = f"/storage/home/yqf5148/work/volvoPennState/Jobs/submit2.sh"  # Assuming different scripts for each job
#     output_file = f"/storage/home/yqf5148/work/volvoPennState/Jobs/log_test/result_{thisVIN}.txt"  # Assuming different output files for each job
#     job_id = submit_job(script_path, thisVIN)

#     if job_id:
#         job_ids.append(job_id)
#         output_files.append(output_file)

# # Check the status of all jobs
# check_jobs_status(job_ids)

# # Retrieve and print outputs for all jobs
# get_jobs_output(output_files)

    # # check if there is any remaining VIN for which we have not generated data. In other words, if  df_all_pop_VINs > df_generated_pop_VINs we should continue generating dsata for the remaining VINs.
    # df_generated_pop_VINs = spark.sql('select * from VINs_data') 
    # df_generated_pop_VINs = df_generated_pop_VINs.toDF("VIN", "TOTAL_ROWS")


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,4V4NC9EH0KN902738,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ACCU,WBD-ACCU,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,4V4NC9EH0KN902738,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ACCU,WBD-ACCU,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,4V4NC9EH0KN902738,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ACCU,WBD-ACCU,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,1M2AX18C9FM027330,D13,425,GU8,WTDF22.5,WTDD24.5,?,WBF-HL,WBD-HL,?,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,1M2AX18C9FM027330,D13,425,GU8,WTDF22.5,WTDD24.5,?,WBF-HL,WBD-HL,?,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57306,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
57307,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
57308,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
57309,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


{'1M2GR2GC7KM001489', '4V4NC9EH3FN923024', '1M2GR2ACXLM001240', '1M1PN4GY9KM001985', '4V4NC9EH2LN244521', '1M2GR2AC2LM001281', '4V4WC9EG7MN273784', '1M1AN2GY4KM012550', '4V4NC9EH0JN886054', '1M2GR2GC3LM013852', '1M2AX18C9FM027330', '4V4W19EG5MN272185', '1M1AW09Y7JM089071', '1M2TE1GC6LM004535', '1M2AX07C3JM037775', '4V4NC9EJ4GN950267', '1M2TE2GC7KM001910', '1M1AN3GY8LM013627', '4V4NC9EH2KN200887', '1M2AU02C2GM010754', '4V4NC9EH4JN885232', '4V4MC9DG2GN960233', '1M2GR4GC6LM014392', '4V4NC9EH0KN902738', '1M1AN4GY3KM006363'}


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,index,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,0,4V4NC9EH0KN902738,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ACCU,WBD-ACCU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1,4V4NC9EH0KN902738,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ACCU,WBD-ACCU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,2,4V4NC9EH0KN902738,D13,455,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ACCU,WBD-ACCU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,3,1M2AX18C9FM027330,D13,425,GU8,WTDF22.5,WTDD24.5,?,WBF-HL,WBD-HL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,4,1M2AX18C9FM027330,D13,425,GU8,WTDF22.5,WTDD24.5,?,WBF-HL,WBD-HL,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57306,57306,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
57307,57307,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
57308,57308,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
57309,57309,1M1AN4GY3KM006363,D13,445,AN6,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [ ]:
display(df_pop_VINs_p)